# SVM Model for Mechanism Prediction - Evaluation & Validation
## Overview

This notebook is based on the original work: "Building Predictive Models for Mechanism-of-Action Classification from Phenotypic Assay Data Sets". The aim is to get a basic working idea of what's going on and get a working model. It is NOT intended to exactly replicate what the original work was doing but rather to achieve similar or better performance. 

The [original work](http://journals.sagepub.com/doi/abs/10.1177/1087057113505324) built a predictive model for assigning mechanism class to compounds and to bioactive agents. The model used 84 features and 309 environmental chemicals. Knowing the mechanism class of chemicals can then be used to evaluate the safety and efficacy of compounds and include classes such as inhibitors of mitochondrial and microtubule function, etc.

## Read the data

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandasql import sqldf


from scripts.random_profiles import RandomProfileGenerator
from scripts.profile_reader import ProfileReader

pd.options.display.max_rows = 5
pd.options.display.max_columns = 300
%matplotlib inline

pqry = lambda q: sqldf(q, globals())

pr = ProfileReader(data_file='data\\Final_Berg JBS 2013 Supplemental Table 3_For SVM14Dec2017.xlsx',  
                       mechanism_file='data\\Final_Berg JBS 2013 Supplemental Table 3_For SVM14Dec2017 - Mechanisms.xlsx')
# read in data. Make column 2-level index, level 0 is system, level 1 ir readout
# row indices are mechanism, agent, concentration
data = pr.parse_profiles()

# Impute missing
grouped = data.groupby(data.index.get_level_values(0).values)
f = lambda x: x.fillna(x.mean())
data = grouped.transform(f)

# convert multiindex columns to flat
data.columns = [':'.join(col).strip() for col in data.columns.values]
data = pqry('''select * from data''')
pqry('''select * from data limit 5''')

,mech,agent,conc,BrEPI_IL-1b/TNF-a/IFN-g_24:CD87/uPAR,BrEPI_IL-1b/TNF-a/IFN-g_24:CXCL10/IP-10,BrEPI_IL-1b/TNF-a/IFN-g_24:CXCL9/MIG,BrEPI_IL-1b/TNF-a/IFN-g_24:HLA-DR,BrEPI_IL-1b/TNF-a/IFN-g_24:IL-1alpha,BrEPI_IL-1b/TNF-a/IFN-g_24:MMP-1,BrEPI_IL-1b/TNF-a/IFN-g_24:PAI-I,BrEPI_IL-1b/TNF-a/IFN-g_24:SRB,BrEPI_IL-1b/TNF-a/IFN-g_24:tPA,BrEPI_IL-1b/TNF-a/IFN-g_24:uPA,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:CD106/VCAM-1,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:Collagen III,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:CXCL10/IP-10,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:CXCL8/IL-8,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:CXCL9/MIG,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:EGFR,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:M-CSF,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:MMP-1,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:PAI-I,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:Proliferation_72hr,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:SRB,HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24:TIMP-2,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:CCL2/MCP-1,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:CD54/ICAM-1,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:CXCL10/IP-10,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:IL-1alpha,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:MMP-9,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:SRB,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:TIMP-2,HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24:uPA,HUVEC/PBMC_LPS_24:CCL2/MCP-1,HUVEC/PBMC_LPS_24:CD106/VCAM-1,HUVEC/PBMC_LPS_24:CD142/Tissue Factor,HUVEC/PBMC_LPS_24:CD40,HUVEC/PBMC_LPS_24:CD62E/E-Selectin,HUVEC/PBMC_LPS_24:CXCL8/IL-8,HUVEC/PBMC_LPS_24:IL-1alpha,HUVEC/PBMC_LPS_24:M-CSF,HUVEC/PBMC_LPS_24:sPGE2,HUVEC/PBMC_LPS_24:SRB,HUVEC/PBMC_LPS_24:sTNF-alpha,HUVEC/PBMC_SEB/TSST_24:CCL2/MCP-1,HUVEC/PBMC_SEB/TSST_24:CD38,HUVEC/PBMC_SEB/TSST_24:CD40,HUVEC/PBMC_SEB/TSST_24:CD62E/E-Selectin,HUVEC/PBMC_SEB/TSST_24:CD69,HUVEC/PBMC_SEB/TSST_24:CXCL8/IL-8,HUVEC/PBMC_SEB/TSST_24:CXCL9/MIG,HUVEC/PBMC_SEB/TSST_24:PBMC Cytotoxicity,HUVEC/PBMC_SEB/TSST_24:Proliferation,HUVEC/PBMC_SEB/TSST_24:SRB,HUVEC_IL-1b/TNF-a/IFN-g_24:CCL2/MCP-1,HUVEC_IL-1b/TNF-a/IFN-g_24:CD106/VCAM-1,HUVEC_IL-1b/TNF-a/IFN-g_24:CD141/Thrombomodulin,HUVEC_IL-1b/TNF-a/IFN-g_24:CD142/Tissue Factor,HUVEC_IL-1b/TNF-a/IFN-g_24:CD54/ICAM-1,HUVEC_IL-1b/TNF-a/IFN-g_24:CD62E/E-Selectin,HUVEC_IL-1b/TNF-a/IFN-g_24:CD87/uPAR,HUVEC_IL-1b/TNF-a/IFN-g_24:CXCL8/IL-8,HUVEC_IL-1b/TNF-a/IFN-g_24:CXCL9/MIG,HUVEC_IL-1b/TNF-a/IFN-g_24:HLA-DR,HUVEC_IL-1b/TNF-a/IFN-g_24:Proliferation,HUVEC_IL-1b/TNF-a/IFN-g_24:SRB,HUVEC_IL-4/Histamine_24:CCL2/MCP-1,HUVEC_IL-4/Histamine_24:CCL26/Eotaxin-3,HUVEC_IL-4/Histamine_24:CD106/VCAM-1,HUVEC_IL-4/Histamine_24:CD62P/P-selectin,HUVEC_IL-4/Histamine_24:CD87/uPAR,HUVEC_IL-4/Histamine_24:SRB,HUVEC_IL-4/Histamine_24:VEGFR2,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CCL2/MCP-1,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CD106/VCAM-1,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CD141/Thrombomodulin,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CD142/Tissue Factor,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CD87/uPAR,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CXCL8/IL-8,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:CXCL9/MIG,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:HLA-DR,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:IL-6,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:LDLR,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:M-CSF,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:Proliferation,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:Serum Amyloid A,CASMC_HCL_IL-1b/TNF-a/IFN-g_24:SRB
0,AhR agonist,FICZ,1000 nM,0.031453,0.020148,-0.006715,-0.027737,-0.022183,0.110827,-0.112386,-0.003285,0.102482,0.018481,0.004346,0.074810,-0.035580,0.002161,-0.008596,0.025059,-0.040733,0.051074,-0.111534,0.048148,-0.038960,0.064136,-0.004362,-0.005137,-0.004074,0.041061,0.009954,-0.001147,0.046371,0.001108,0.009104,-0.018277,0.155624,0.015812,-0.016767,0.006921,0.096784,-0.013816,0.028806,-0.028511,-0.139501,-0.002186,-0.017793,0.009821,0.116732,-0.024614,0.131661,-0.008077,-0.012467,-0.044135,-0.018509,-0.012472,-0.018993,0.020808,0.142367,0.034303,0.020455,0.014791,0.087453,-0.008754,-0.013645,-0.088930,-0.019053,0.023189,-0.219655,-0.043045,-0.077100,0.000649,-0.040823,0.043608,-0.013293,-0.016680,0.125897,0.016830

## Features
Each feature label is the cell type + stimulation (System) name combined with the readout name (Readout), separated by a colon. 

In [6]:
sm = pd.DataFrame([s.split(':') for s in data.columns[2:].values.astype(str) ])
q = sm.groupby([0,1]).count()
q.index.names=['System','Readout']
q

Empty DataFrame
Columns: []
Index: [(BrEPI_IL-1b/TNF-a/IFN-g_24, CD87/uPAR), (BrEPI_IL-1b/TNF-a/IFN-g_24, CXCL10/IP-10), (BrEPI_IL-1b/TNF-a/IFN-g_24, CXCL9/MIG), (BrEPI_IL-1b/TNF-a/IFN-g_24, HLA-DR), (BrEPI_IL-1b/TNF-a/IFN-g_24, IL-1alpha), (BrEPI_IL-1b/TNF-a/IFN-g_24, MMP-1), (BrEPI_IL-1b/TNF-a/IFN-g_24, PAI-I), (BrEPI_IL-1b/TNF-a/IFN-g_24, SRB), (BrEPI_IL-1b/TNF-a/IFN-g_24, tPA), (BrEPI_IL-1b/TNF-a/IFN-g_24, uPA), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CCL2/MCP-1), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CD106/VCAM-1), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CD141/Thrombomodulin), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CD142/Tissue Factor), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CD87/uPAR), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CXCL8/IL-8), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, CXCL9/MIG), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, HLA-DR), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, IL-6), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, LDLR), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, M-CSF), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, Proliferation), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, SRB), (CASMC_HCL_IL-1b/TNF-a/IFN-g_24, Serum Amyloid A), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, CD106/VCAM-1), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, CXCL10/IP-10), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, CXCL8/IL-8), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, CXCL9/MIG), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, Collagen III), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, EGFR), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, M-CSF), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, MMP-1), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, PAI-I), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, Proliferation_72hr), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, SRB), (HDFn_IL-1b/TNF-a/IFN-g/EGF/FGF/PDGFbb_24, TIMP-2), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, CCL2/MCP-1), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, CD54/ICAM-1), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, CXCL10/IP-10), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, IL-1alpha), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, MMP-9), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, SRB), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, TIMP-2), (HEK/HDFn_IL-1b/TNF-a/IFN-g/TGF-b_24, uPA), (HUVEC/PBMC_LPS_24, CCL2/MCP-1), (HUVEC/PBMC_LPS_24, CD106/VCAM-1), (HUVEC/PBMC_LPS_24, CD142/Tissue Factor), (HUVEC/PBMC_LPS_24, CD40), (HUVEC/PBMC_LPS_24, CD62E/E-Selectin), (HUVEC/PBMC_LPS_24, CXCL8/IL-8), (HUVEC/PBMC_LPS_24, IL-1alpha), (HUVEC/PBMC_LPS_24, M-CSF), (HUVEC/PBMC_LPS_24, SRB), (HUVEC/PBMC_LPS_24, sPGE2), (HUVEC/PBMC_LPS_24, sTNF-alpha), (HUVEC/PBMC_SEB/TSST_24, CCL2/MCP-1), (HUVEC/PBMC_SEB/TSST_24, CD38), (HUVEC/PBMC_SEB/TSST_24, CD40), (HUVEC/PBMC_SEB/TSST_24, CD62E/E-Selectin), (HUVEC/PBMC_SEB/TSST_24, CD69), (HUVEC/PBMC_SEB/TSST_24, CXCL8/IL-8), (HUVEC/PBMC_SEB/TSST_24, CXCL9/MIG), (HUVEC/PBMC_SEB/TSST_24, PBMC Cytotoxicity), (HUVEC/PBMC_SEB/TSST_24, Proliferation), (HUVEC/PBMC_SEB/TSST_24, SRB), (HUVEC_IL-1b/TNF-a/IFN-g_24, CCL2/MCP-1), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD106/VCAM-1), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD141/Thrombomodulin), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD142/Tissue Factor), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD54/ICAM-1), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD62E/E-Selectin), (HUVEC_IL-1b/TNF-a/IFN-g_24, CD87/uPAR), (HUVEC_IL-1b/TNF-a/IFN-g_24, CXCL8/IL-8), (HUVEC_IL-1b/TNF-a/IFN-g_24, CXCL9/MIG), (HUVEC_IL-1b/TNF-a/IFN-g_24, HLA-DR), (HUVEC_IL-1b/TNF-a/IFN-g_24, Proliferation), (HUVEC_IL-1b/TNF-a/IFN-g_24, SRB), (HUVEC_IL-4/Histamine_24, CCL2/MCP-1), (HUVEC_IL-4/Histamine_24, CCL26/Eotaxin-3), (HUVEC_IL-4/Histamine_24, CD106/VCAM-1), (HUVEC_IL-4/Histamine_24, CD62P/P-selectin), (HUVEC_IL-4/Histamine_24, CD87/uPAR), (HUVEC_IL-4/Histamine_24, SRB), (HUVEC_IL-4/Histamine_24, VEGFR2)]

[84 rows x 0 columns]

----
## Data Exploration
We begin with looking at one mechanism: mTOR Inhibitor. 

### Feature observations
Each feature in the data set is a biomarker readout. The features are labeled as System:Readout. The set of Readouts together make up Compound Profiles that are generated by measuring changes in the levels of a set (8–12) of biomarkers (proteins, mediators, known disease risk factors, etc.) in each System. These features indicate a response of a cell type (System) to a certain drug by making more or less of each type of Readout measured. 
We hope that the response of a cell to a drug class manifests itself in the pattern it exhibits in the profile. The profile is a vector of continuous value that represents a point in 84 dimensional space, each readout being a dimension. The values are themselves the logarithm of an n-fold increase or decrease in a expression levels of biomarkers. Their range is typically -2 to + 2.

### Calculate Statistics
[this notebook](Profile%20Statistics.ipynb). 

### Profile plots
The plots all data points grouped by mechanisms class can be seen in this [this notebook](Profile%20Plotting%20for%20each%20Mechanims.ipynb). 

### Mechanism box plots
The visualization of readout value distribution as box plots can be seen in [this notebook](Box%20plots%20of%20mechanism%20classes.ipynb). 



### Generating random profiles to use as negative class
You can see the plots of negative profiles [in this notebook](Print%20Random%20Profiles.ipynb)

----
## Develop Model

### Define performance metric
It is important to establish a single numerical measure to evaluate how well our model performs. F-beta score is a good measure that takes both Precision and Recall into consideration. Choice of 0.5 for beta rewards higher accuracy. 
	Precision (P):  TP/(TP + FP)
	Recall (R): TP / (TP + FN)
	(Precision is referred to by the original paper as PPV)
	F0.5 places more 0.5 time more importance to recall, or 2x time more importance to precision.
	
$F_\beta = (1 + \beta^2) * \frac{precision * recall}{(\beta^2 * precision) + recall}$
    
$F_{0.25} = (1.25) * \frac{precision * recall}{(0.25 * precision) + recall}$
	
From <https://en.wikipedia.org/wiki/Precision_and_recall> 

For the purposes of comparing with the original work, PPV will be calculated, too.

### Does the model capture differences among classes?

----
## Optimize Model

### Grid Search
 - Grid search is used to find the optimal values of a learning algorithm. It is used to calculate many performace scores of a learning algorithm corresponding to a range of hyper-parameter values and pick the best set of hyper-parameter values. It is a 'grid' in the sense that permutations of hyper-parameter values calculated that can be summarized and displayed in grid or a table.
 - Grid search can be applied to optimize a learning algorithm by running it with a range of hyper-parameters to see which combination of parameters perform best. A set of hyper-parameretes are varied between predefined ranges of values and the model is trained with these values. The optimal set of hyper-parameters are then reported. The model is then tested (evaluated) using a portion of the data not used for training.
 - The hyper parameter of importance when working with SVM is C. To balance margin violations and keeping the separation as wide as possible, SVM algorithm has the hyper-parameter C to control this balance. Reducing C will generalize the model better because it emphasizes the regularization term. Large value of C gives higher weight to the individual feaures while therefore it increases variance. 

----
## Analyzing Model Performance
This section we'll take a look at the model's learning and testing performances on subsets of training data. Graphing the model's performance based on hyperparameter "C" and the number of training points can reveal details that may not have been apparent from the results alone.

### Learning Curves
This graph visalizes the SVM both model's training and testing performance with increasing data set size. The shaded region denotes uncertainty of the curve measured in standard deviation.
The model is scored on both the training and testing sets using F1, the coefficient of determination.



-----
## Evaluating Model Performance
We'll construct a model and make a prediction on the compounds in client data sets, using the optimized model.

### Grid Search
 - Grid search is used to find the optimal values of a learning algorithm. It is used to calculate many performace scores of a learning algorithm corresponding to a range of hyper-parameter values and pick the best set of hyper-parameter values. It is a 'grid' in the sense that permutations of hyper-parameter values calculated that can be summarized and displayed in grid or a table.
 - Grid search can be applied to optimize a learning algorithm by running it with a range of hyper-parameters to see which combination of parameters perform best. A set of hyper-parameretes are varied between predefined ranges of values and the model is trained with these values. The optimal set of hyper-parameters are then reported. The model is then tested (evaluated) using a portion of the data not used for training.
 - The hyper parameter of importance when working with SVM is C. To balance margin violations and keeping the separation as wide as possible, SVM algorithm has the hyper-parameter C to control this balance. Reducing C will generalize the model better because it emphasizes the regularization term. Large value of C gives higher weight to the individual feaures while therefore it increases variance. 

----
## Making Predictions
The SVM model has been trained on the given set of data, it can now be used to make predictions on new sets of input compounds. We can use these prediciond to gain information about a compound exhibits featues that make it likely to be in a Mechanism class.
